In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, interactive_output, IntSlider, FloatSlider, fixed


In [2]:
# Task 1

def laguerre(t, n, beta=2, sigma=4):
    if beta > sigma or beta < 0:
        raise ValueError("Wrong beta and sigma")
    
    l_0 = np.sqrt(sigma) * (np.exp(-beta * t / 2))
    l_1 = np.sqrt(sigma) * (1 - sigma * t) * (np.exp(-beta * t / 2))

    if n == 0:
        return l_0
    elif n == 1:
        return l_1
    
    l_next = (3 - t * sigma) / 2 * l_1 - 0.5 * l_0
    for j in range(3, n+1):
        l_0 = l_1
        l_1 = l_next
        l_next = (2 * j - 1 - t * sigma) / j * l_1 - (j - 1) / j * l_0
    return l_next

In [3]:
widgets.interact(laguerre, t=(0.0, 5.0, 0.1), n=(0, 10, 1), beta=(0, 6, 0.1), sigma=(0, 6, 0.1))

interactive(children=(FloatSlider(value=2.5, description='t', max=5.0), IntSlider(value=5, description='n', ma…

<function __main__.laguerre(t, n, beta=2, sigma=4)>

In [4]:
# Task2
def tabulate_laguerre(n, T, beta, sigma):
    steps = np.linspace(0, T, 100)
    res = laguerre(steps, n, beta,sigma)
    res_df = pd.DataFrame({"tab": steps, "result": res})

    return res_df

widgets.interact(tabulate_laguerre, n=(0, 10, 1), T=(1, 10, 1), beta=(0, 4, 0.1), sigma=(0, 4, 0.1))


interactive(children=(IntSlider(value=5, description='n', max=10), IntSlider(value=5, description='T', max=10,…

<function __main__.tabulate_laguerre(n, T, beta, sigma)>

In [5]:
# Task 3
def experiment(beta=2, sigma=4, eps=0.001):
    N = 20
    t = 0
    while True:
        t += 0.001
        res = []
        flag = True
        for i in range(N):
            val = abs(laguerre(t, i, beta, sigma))
            if val > eps:
                flag = False
                break
            res.append(val)
        if flag:
            return t, pd.DataFrame({"n": range(N), "res": res})

In [6]:
widgets.interact(experiment, beta=(0,10,0.1), sigma=(0,10,0.1), eps=(0, 0.001, 0.0001))

interactive(children=(FloatSlider(value=2.0, description='beta', max=10.0), FloatSlider(value=4.0, description…

<function __main__.experiment(beta=2, sigma=4, eps=0.001)>

In [7]:
# Task 4
def rects_integral(f, T, N=1000):
    step = T / N
    integral = 0

    for i in range(N):
        t = i * step
        integral += f(t) * step

    return integral


In [8]:
f1= lambda t: np.exp(-2*t**3/3)
print(rects_integral(f1, 100,100))
print(rects_integral(f1, 100,10000))
print(rects_integral(f1, 100, 1000))
print(rects_integral(f1, 500, 250))

1.5182450842564033
1.0272063651460819
1.072205809646119
2.009655899987663


In [9]:
# Task 5
def laguerre_transformation(f, n, T, beta=2, sigma=4):
    alpha = beta - sigma
    integrand = lambda t: f(t) * laguerre(t, n) * np.exp(-t*alpha)
    result = rects_integral(integrand, T)
    return result

In [10]:
def f(t):
    if 0 <= t <= 2 * np.pi:
        return np.sin(t - np.pi / 2) + 1
    elif t > 2 * np.pi:
        return 0

In [11]:
def tab_transformation(f, T, N=20, beta=2, sigma=4):
    t = range(1, N+1)
    results = [laguerre_transformation(f, n, T, beta, sigma) for n in t]
    return results

In [12]:
widgets.interact(tab_transformation, f=widgets.fixed(f), T=(1,100,1), N=(1,30,1), beta=(0,10,0.1), sigma=(0,10,0.1))

interactive(children=(IntSlider(value=50, description='T', min=1), IntSlider(value=20, description='N', max=30…

<function __main__.tab_transformation(f, T, N=20, beta=2, sigma=4)>

In [13]:
# Task6
def reverse_laguerre_transformation(lst, t, beta=2 , sigma=4):
    return sum([lst[i] * laguerre(t, i) for i in range(len(lst))])

In [14]:
def f_t(t):
    return t

In [15]:
lst = tab_transformation(f, 100, 2, 4)
widgets.interact(reverse_laguerre_transformation, lst=widgets.fixed(lst), t=(0,10,1), beta=widgets.fixed(2), sigma=widgets.fixed(4))

interactive(children=(IntSlider(value=5, description='t', max=10), Output()), _dom_classes=('widget-interact',…

<function __main__.reverse_laguerre_transformation(lst, t, beta=2, sigma=4)>

In [16]:
# Task 7
def plot_laguerre(T, N, beta=2, sigma=4):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    for n in range(N+1):
        values = tabulate_laguerre(n, T, beta, sigma)
        ax.plot(values['tab'], values['result'], label=f"n={n}")
    
    ax.set_xlabel("t")
    ax.set_ylabel("l(t)")
    fig.legend(loc='lower center', ncol=4)
    plt.show()


In [17]:

widgets.interact(plot_laguerre, T=(0.0, 20.0, 0.25), N=(0, 30, 1), beta=(0, 4, 0.1), sigma=(0, 4, 0.1))

interactive(children=(FloatSlider(value=10.0, description='T', max=20.0, step=0.25), IntSlider(value=15, descr…

<function __main__.plot_laguerre(T, N, beta=2, sigma=4)>

In [18]:
# Task 8
def plot_transformation(f, n, T, beta=2, sigma=4):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    values = tab_transformation(f, T, n, beta, sigma)
    ax.bar(range(1, n+1), values)

    ax.set_xlabel("n")
    ax.set_ylabel("f_n")
    ax.set_title("Laguerre Transformation")
    plt.axhline(0, color='black')
    plt.show()

In [19]:
widgets.interact(plot_transformation, f=widgets.fixed(f), T=(0.0, 20.0, 0.25), n=(0, 30, 1), beta=(0, 6, 0.1), sigma=(0, 6, 0.1))

interactive(children=(IntSlider(value=15, description='n', max=30), FloatSlider(value=10.0, description='T', m…

<function __main__.plot_transformation(f, n, T, beta=2, sigma=4)>

In [20]:
def plot_tranformations(f, n, T, beta=2, sigma=4, t1=0, t2=2*np.pi):

    transform_values = tab_transformation(f, T, n, beta, sigma)
    reversed_transform_values = [reverse_laguerre_transformation(transform_values, t, beta, sigma) for t in np.linspace(t1, t2, 1000)]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.subplots(2, 1)
    ax[0].bar(range(1, n+1), transform_values, alpha=0.7, edgecolor='black')

    ax[0].set_xlabel("n")
    ax[0].set_ylabel("f_n")
    ax[0].set_title("Transformation coefs")
    ax[0].set_xticks(range(1, n+1))
    fig.tight_layout()
    ax[0].axhline(0, color='black')

    ax[1].plot(np.linspace(t1, t2, 1000), reversed_transform_values, alpha=0.7, linewidth=2.0)
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("f(t)")
    ax[1].set_title("Reversed transformation")

    plt.show()

In [21]:
widgets.interact(plot_tranformations, f=widgets.fixed(f),  T=(0.0, 20.0, 0.25), n=(0, 30, 1), beta=(0, 6, 0.1), sigma=(0, 6, 0.1), t1=(-5*np.pi, 5*np.pi, np.pi), t2=(-5*np.pi, 5*np.pi, np.pi))

interactive(children=(IntSlider(value=15, description='n', max=30), FloatSlider(value=10.0, description='T', m…

<function __main__.plot_tranformations(f, n, T, beta=2, sigma=4, t1=0, t2=6.283185307179586)>